In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from scipy import stats
import pickle
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
class MySymRV(stats.rv_continuous):
    def __init__(self, name="SymMix", shapes="alpha, sc1, mu2, sc2", **args):
        super(MySymRV, self).__init__(shapes=shapes, name=name, **args)

    def __call__(self, *args, **kwds):
        mu2 = np.sqrt((kwds["gvar"] + (kwds["alpha"]-1)*(kwds["sc1"]**2) - kwds["alpha"]*(kwds["sc2"]**2))/kwds["alpha"])
        print(mu2)
        kwds["mu2"] = mu2
        del kwds["gvar"]
        return super(MySymRV, self).__call__(*args, **kwds)

    def _pdf(self, x, alpha, sc1, mu2, sc2,*args, **kwargs):
        p1 = (1-alpha) * stats.norm.pdf(x, loc=0, scale=sc1)
        p2 = alpha/2 * stats.norm.pdf(x, loc=-mu2, scale=sc2)
        p3 = alpha/2 * stats.norm.pdf(x, loc=mu2, scale=sc2)
        return p1+p2+p3
    def _cdf(self, x, alpha, sc1, mu2, sc2,*args, **kwargs):
        p1 = (1-alpha) * stats.norm.cdf(x, loc=0, scale=sc1)
        p2 = alpha/2 * stats.norm.cdf(x, loc=-mu2, scale=sc2)
        p3 = alpha/2 * stats.norm.cdf(x, loc=mu2, scale=sc2)
        return p1+p2+p3
    def _rvs(self, alpha, sc1, mu2, sc2, *args, **kwargs):
        x = np.prod(self._size)
        vect = np.zeros(x)
        sxp = stats.multinomial.rvs(x, [(1-alpha), alpha/2, alpha/2], size=1)[0]
        if sxp[0]>0:
            vect[:sxp[0]] = stats.norm.rvs(loc=0, scale=sc1, size=sxp[0])
        if sxp[1]>0:
            vect[sxp[0]:np.sum(sxp[:2])] = stats.norm.rvs(loc=-mu2, scale=sc2, size=sxp[1])
        if sxp[2]>0:
            vect[np.sum(sxp[:2]):x] = stats.norm.rvs(loc=mu2, scale=sc2, size=sxp[2])
        np.random.shuffle(vect)
        vect = vect.reshape(self._size)
        return vect
    def title(self, alpha, sc1, mu2, sc2):
        aux = str(np.round(alpha/2,4))+"*N(-"+str(np.round(mu2,3))+","+str(np.round(sc2**2,3))+")+"+\
          str(np.round(1-alpha,4))+"*N(0,"+str(np.round(sc1**2,3))+")+"+\
          str(np.round(alpha/2,4))+"*N("+str(np.round(mu2,3))+","+str(np.round(sc2**2,3))+")"
        return aux

In [ ]:
class MyAsymRV(stats.rv_continuous):
    def __init__(self, name="AsymMix", shapes="alpha, mu1, sc1, mu2, sc2", **args):
        super(MyAsymRV, self).__init__(shapes=shapes, name=name, **args)

    def __call__(self, *args, **kwds):
        #sc2 = kwds["gvar"] - (1-kwds["alpha"])*(kwds["sc1"]**2 + mu1**2) - kwds["alpha"]*(kwds["mu2"]**2) # - gmu**2
        #sc2 = np.sqrt(sc2/kwds["alpha"])
        mu2 = np.sqrt((1-kwds["alpha"])/(kwds["alpha"])*\
                      (kwds["gvar"] + (kwds["alpha"]-1)*(kwds["sc1"]**2) - kwds["alpha"]*(kwds["sc2"]**2)))
        mu1 = mu2 * kwds["alpha"] / (kwds["alpha"]-1) # to fix mean=0
        print(mu1,mu2)
        #kwds["sc2"] = sc2
        kwds["mu1"] = -mu1
        kwds["mu2"] = mu2
        del kwds["gvar"]
        return super(MyAsymRV, self).__call__(*args, **kwds)
        
    def _pdf(self, x, alpha, mu1, sc1, mu2, sc2,*args, **kwargs):
        p1 = (1-alpha) * stats.norm.pdf(x, loc=-mu1, scale=sc1)
        p2 = alpha * stats.norm.pdf(x, loc=mu2, scale=sc2)
        return p1+p2
    def _cdf(self, x, alpha, mu1, sc1, mu2, sc2,*args, **kwargs):
        p1 = (1-alpha) * stats.norm.cdf(x, loc=-mu1, scale=sc1)
        p2 = alpha * stats.norm.cdf(x, loc=mu2, scale=sc2)
        return p1+p2
    def _rvs(self, alpha, mu1, sc1, mu2, sc2, *args, **kwargs):
        x = np.prod(self._size)
        vect = np.zeros(x)
        sxp = stats.multinomial.rvs(x, [(1-alpha), alpha], size=1)[0]
        if sxp[0]>0:
            vect[:sxp[0]] = stats.norm.rvs(loc=-mu1, scale=sc1, size=sxp[0])
        if sxp[1]>0:
            vect[sxp[0]:] = stats.norm.rvs(loc=mu2, scale=sc2, size=sxp[1])
        np.random.shuffle(vect)
        vect = vect.reshape(self._size)
        return vect
    def title(self, alpha, mu1, sc1, mu2, sc2):
        aux = str(np.round(1-alpha,4))+"*N(-"+str(np.round(mu1,3))+","+str(np.round(sc1**2,3))+")+"+\
          str(np.round(alpha,4))+"*N("+str(np.round(mu2,3))+","+str(np.round(sc2**2,3))+")"
        return aux

In [ ]:
# Params
rv1_global_var = 6.0
rv1_main_var = 0.2
rv1_extr_var = 1.0
rv2_global_var = 4.0
alpha = 2.**(-10)

mixture = MyAsymRV

In [ ]:
def mse(estimates, real_val=0):
    return np.mean((real_val-estimates)**2)
def mare(estimates, real_val=1):
    return np.mean(np.abs((real_val-estimates)/real_val))
def mae(estimates, real_val=0):
    return np.mean(np.abs(real_val-estimates))

In [ ]:
# Obtains n_estimates number of estimates from an estimator that is the mean of n_samples samples.
def get_mse_mae_with_estimates(rv1, rv2, n_estimates, n_samples, real_val=0):
    samples1 = rv1.rvs(size=(n_estimates,n_samples))
    samples2 = rv2.rvs(size=(n_estimates,n_samples))
    estimates1 = np.mean(samples1,axis=1)
    estimates2 = np.mean(samples2,axis=1)
    mse1 = mse(estimates1)
    mse2 = mse(estimates2)
    mae1 = mae(estimates1)
    mae2 = mae(estimates2)
    return int(mse1<=mse2), int(mae1<=mae2)

In [ ]:

# We repeat each experiment n_repetitions time 
# In each experiment we compare the MSE of repeating an estimator n_estimates times
# Each estimator is the average of n_samples samples

n_reps = 1000
n_pnts = 12

l_alphas = 2.0**(-1*np.arange(1,n_pnts))
l_num_samples = 2**np.arange(n_pnts)
l_num_estimates = 2**np.arange(n_pnts)

results_mse = np.zeros((len(l_alphas),len(l_num_samples),len(l_num_estimates)))
results_mae = np.zeros((len(l_alphas),len(l_num_samples),len(l_num_estimates)))

my_rv2 = stats.norm(loc=0, scale=np.sqrt(rv2_global_var))

for i_a, alpha in enumerate(l_alphas):
    print("alpha",i_a,alpha)
    my_rv1 = mixture()
    my_rv1 = my_rv1(alpha=alpha, sc1=np.sqrt(rv1_main_var), sc2=np.sqrt(rv1_extr_var), gvar=rv1_global_var)
    #my_rv1 = my_rv1(sc1=rv1_main_stdev, alpha=alpha, mu2=rv1_extr_mean, sc2=rv1_extr_stdev)

    for i_s, n_samples in enumerate(l_num_samples):
        print("samples",i_s,n_samples)
        for i_e, n_estimates in enumerate(l_num_estimates):
            print("estimates",i_e,n_estimates)
            counts = np.zeros((n_reps,2))
            for i in np.arange(n_reps):
                counts[i,:] = get_mse_mae_with_estimates(my_rv1, my_rv2, n_estimates, n_samples, real_val=0)
            aux = np.mean(counts,axis=0)
            results_mse[i_a,i_s,i_e] = aux[0]
            results_mae[i_a,i_s,i_e] = aux[1]

In [ ]:
f_pckl_name = "results/exp200505_"+my_rv1.dist.name+"_estim_analysis"+ \
    "_nrep_"+str(n_reps)+"_npt_"+str(n_pnts)+"_varA_"+str(rv2_global_var) + \
    "_varB_"+str(rv1_global_var) +"_varB1_"+str(rv1_main_var) +"_varB2_"+str(rv1_extr_var)+".pickle"
data = {"mse":results_mse,
        "mae":results_mae}

with open(f_pckl_name, 'wb') as f_pckl:
    pickle.dump(data, f_pckl, pickle.HIGHEST_PROTOCOL)


## Plot distributions

In [ ]:
# Params
#rv1_global_var = 6.0
#rv1_main_var = 0.2
#rv1_extr_var = 1.0
#rv2_global_var = 4.0
alpha = 2.**(-10)

mixture = MyAsymRV

In [ ]:
my_rv1 = mixture()
my_rv1 = my_rv1(alpha=alpha, sc1=np.sqrt(rv1_main_var), sc2=np.sqrt(rv1_extr_var), gvar=rv1_global_var)
var1 = my_rv1.var()
my_rv2 = stats.norm(loc=0, scale=np.sqrt(rv2_global_var))
var2 = my_rv2.var()
print("var rv1",var1)
print("var rv2",var2)

In [ ]:
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes, inset_axes, mark_inset

X = np.linspace(-np.round(my_rv1.kwds["mu2"]*1.1), np.round(my_rv1.kwds["mu2"]*1.1), 1000)
pdf1 = my_rv1.pdf(X)
pdf2 = my_rv2.pdf(X)
fig,ax = plt.subplots(figsize=(10,4))
ax.set_title("Two distributions, a normal N(0,"+str(rv2_global_var)+") and a mixture " + my_rv1.dist.title(**my_rv1.kwds))
ax.plot(X, pdf1, label=my_rv1.dist.name+' (Var.= '+str(np.round(var1,4))+")")
ax.plot(X, pdf2, label='Normal (Var.= '+str(np.round(var2,4))+")")
ax.legend()#loc='center left')

axins = inset_axes(ax, 1.5,.7 , loc=1, bbox_to_anchor=(0.85, 0.7), bbox_transform=ax.figure.transFigure) # no zoom

x1, x2 = my_rv1.kwds["mu2"]-my_rv1.kwds["sc2"]*4, my_rv1.kwds["mu2"]+my_rv1.kwds["sc2"]*4#np.round(my_rv1.kwds["mu2"]*.9),np.round(my_rv1.kwds["mu2"]*1.1)
alt_X = np.linspace(x1, x2, 1000)
alt_pdf1 = my_rv1.pdf(alt_X)
alt_pdf2 = my_rv2.pdf(alt_X)
axins.plot(alt_X, alt_pdf1)
axins.plot(alt_X, alt_pdf2)

val = np.max(alt_pdf1)
y1, y2 = -val*0.2, val+val*0.2 # specify the limits
axins.set_xlim(x1, x2) # apply the x-limits
axins.set_ylim(y1, y2) # apply the y-limits

mark_inset(ax, axins, loc1=4, loc2=3, fc="none", ec="0.5")

if my_rv1.dist.name=="SymMix":
    axins = inset_axes(ax, 4,2 , loc=1, bbox_to_anchor=(0.4, 0.7), bbox_transform=ax.figure.transFigure) # no zoom

    x1, x2 = -my_rv1.kwds["mu2"]-my_rv1.kwds["sc2"]*4, -my_rv1.kwds["mu2"]+my_rv1.kwds["sc2"]*4#np.round(-my_rv1.kwds["mu2"]*1.1),np.round(-my_rv1.kwds["mu2"]*.9)
    alt_X = np.linspace(x1, x2, 1000)
    alt_pdf1 = my_rv1.pdf(alt_X)
    alt_pdf2 = my_rv2.pdf(alt_X)
    axins.plot(alt_X, alt_pdf1)
    axins.plot(alt_X, alt_pdf2)

    val = np.max(alt_pdf1)
    y1, y2 = -val*0.2, val+val*0.2 # specify the limits
    axins.set_xlim(x1, x2) # apply the x-limits
    axins.set_ylim(y1, y2) # apply the y-limits

    mark_inset(ax, axins, loc1=2, loc2=4, fc="none", ec="0.5")

plt.show()

## Print results

In [ ]:
file = "results/exp200505_AsymMix_estim_analysis_nrep_1000_npt_12_varA_4.0_varB_6.0_varB1_0.2_varB2_1.0.pickle"
with open(file, 'rb') as f_pckl:
    data = pickle.load(f_pckl)


In [ ]:
n_pnts = 12
results_mse = data["mse"]

In [ ]:
X = np.arange(n_pnts)
Y = np.arange(n_pnts)
X, Y = np.meshgrid(X, Y)

Za = results_mse[10,:,:]
Zb = results_mse[9,:,:]

In [ ]:
fig = plt.figure(figsize=(10,7))
ax = fig.gca(projection='3d')

surf = ax.plot_surface(X, Y, Za, cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)

#surf = ax.plot_surface(X, Y, Zb, cmap=cm.cool,
#                       linewidth=0, antialiased=False)

ax.set_xlabel("Num. estimators")
ax.set_ylabel("Num. samples")
ax.set_zlabel("Proportion of cases Err(LowVar) >= Err(LargeVar)")

ax.set_xticks(np.arange(1,n_pnts,2))
ax.set_xticklabels((2.**np.arange(1,n_pnts,2)).astype(int))

ax.set_yticks(np.arange(1,n_pnts,2))
ax.set_yticklabels((2.**np.arange(1,n_pnts,2)).astype(int))

ax.view_init(elev=15, azim=115)
# Add a color bar which maps values to colors.
#fig.colorbar(surf, shrink=0.5, aspect=5)

plt.show()


In [ ]:
fig, ax = plt.subplots(figsize=(7,5))

X = np.arange(n_pnts)
Y = results_mse[10,0,:]

ax.plot(X, Y)
ax.set_xticks(np.arange(1,n_pnts,2))
ax.set_xticklabels((2.**np.arange(1,n_pnts,2)).astype(int))

ax.set_xlabel("Num. estimators")

ax.set_ylim((0,1))
ax.set_ylabel("Proportion of cases Err(LowVar) >= Err(LargeVar)")

plt.show()